### Data set contains train set and test set with two classes cars and planes.Predicting class using car or plane

## Unzipping the data

In [ ]:
#!unzip v_data.zip

## Imported all needed packages

In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import load_img
from sklearn import svm

## Reading images and appending to arrays

In [ ]:
Categories=['cars','planes']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='v_data/train' 
for i in Categories:
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(224,224,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))


## Coverting to numpy arrays and intializing X and Y

In [ ]:
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
x=df.iloc[:,:-1] 
y=df.iloc[:,-1] 
print(x.head(5))
print(y.head(5))

     0         1         2       ...    150525    150526    150527
0  0.996078  1.000000  1.000000  ...  1.000000  1.000000  0.984314
1  0.094118  0.113725  0.090196  ...  0.109804  0.121569  0.086275
2  0.105882  0.105882  0.113725  ...  0.352941  0.356863  0.364706
3  0.866667  0.831373  0.701961  ...  0.960784  0.917647  0.839216
4  0.462745  0.545098  0.658824  ...  0.337255  0.337255  0.329412

[5 rows x 150528 columns]
0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64


## Intializing variables for CNN

In [ ]:
train_data_dir = 'v_data/train'
validation_data_dir = 'v_data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

## Creating a model and adding layers to the model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

## Genetraing the data from the directories and fitting to the cnn model

In [ ]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
25/25 [==============================] - 20s 808ms/step - loss: 0.5474 - accuracy: 0.7375 - val_loss: 0.3875 - val_accuracy: 0.8854
Epoch 2/10
25/25 [==============================] - 20s 803ms/step - loss: 0.4802 - accuracy: 0.7775 - val_loss: 0.3106 - val_accuracy: 0.9062
Epoch 3/10
25/25 [==============================] - 20s 808ms/step - loss: 0.4278 - accuracy: 0.8125 - val_loss: 0.4719 - val_accuracy: 0.7812
Epoch 4/10
25/25 [==============================] - 21s 816ms/step - loss: 0.4224 - accuracy: 0.8075 - val_loss: 0.3151 - val_accuracy: 0.8750
Epoch 5/10
25/25 [==============================] - 20s 800ms/step - loss: 0.3604 - accuracy: 0.8375 - val_loss: 0.2660 - val_accuracy: 0.8750
Epoch 6/10
25/25 [==============================] - 20s 802ms/step - loss: 0.3499 - accuracy: 0.8475 - val_loss: 0.3788 - val_accuracy: 0.8333
Epoch 7/10
25/25 [==============================] - 20s 799ms/step - loss: 0.3521 - accuracy: 0.8550 - val_loss: 0.3820 - val_accuracy: 0.8438

## Prediction of class using CNN model

In [ ]:
image = load_img('v_data/test/cars/5.jpg', target_size=(227, 227))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,227,227,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", np.argmax(label,axis=1))

Predicted Class (0 - Cars , 1- Planes):  [0]


## Creating Svm model

In [ ]:
svc=svm.SVC()
svc.fit(x,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Prediction of class using SVM model

In [ ]:
img_array=imread('v_data/test/cars/5.jpg')
img_resized=resize(img_array,(224,224,3))
img=img_resized.flatten()
img1=np.array([img])
p=svc.predict(img1)
print("Predicted Class (0 - Cars , 1- Planes): ", p)

Predicted Class (0 - Cars , 1- Planes):  [0]
